In [1]:
from __future__ import print_function

import numpy as np
import warnings

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.imagenet_utils import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image
import tensorflow as tf

In [2]:
with tf.device('GPU:1'):
    def VGGupdated(input_tensor=None,classes=2):    
   
        img_rows, img_cols = 224,224
        img_channels = 3

        img_dim = (img_rows, img_cols, img_channels)
    
        img_input = Input(shape=img_dim)
        
        # Block 1
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
        x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

        # Block 2
        x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
        x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

        # Block 3
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
        x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

        # Block 4
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

        # Block 5
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
        x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

        
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)

        # Create model.
        model = Model(inputs = img_input, outputs = x, name='VGGdemo')
        return model

In [3]:
model = VGGupdated(classes = 2) #animal abuse and not animal abuse

In [4]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('dataset')

data = os.listdir('dataset')
print (data)  

print("data: ", len(dataset_path))

['Animal_Abuse', 'NOT_ANIMAL_ABUSE']
data:  2


In [6]:
import os
import pandas as pd

# Path to the main folder containing class folders
main_folder = "dataset"

# List to store class names and image paths
data = []

# Loop through each class folder
for class_name in os.listdir(main_folder):
    class_folder = os.path.join(main_folder, class_name)
    
    if os.path.isdir(class_folder):
        for img_file in os.listdir(class_folder):
            if img_file.endswith(".jpg") or img_file.endswith(".png"):
                img_path = os.path.join(class_folder, img_file)
                data.append((class_name, img_path))

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=["class_name", "img_path"])
df

,class_name,img_path
0,Animal_Abuse,dataset\Animal_Abuse\07955b72df.jpg
1,Animal_Abuse,dataset\Animal_Abuse\1.jpg
2,Animal_Abuse,dataset\Animal_Abuse\10.jpg
3,Animal_Abuse,dataset\Animal_Abuse\100.jpg
4,Animal_Abuse,dataset\Animal_Abuse\1000.jpg
...,...,...
9216,NOT_ANIMAL_ABUSE,dataset\NOT_ANIMAL_ABUSE\bf910ea125.jpg
9217,NOT_ANIMAL_ABUSE,dataset\NOT_ANIMAL_ABUSE\fab6e81536.jpg
9218,NOT_ANIMAL_ABUSE,dataset\NOT_ANIMAL_ABUSE\fc1dd26858.jpg
9219,NOT_ANIMAL_ABUSE,dataset\NOT_ANIMAL_ABUSE\fc6545699d.jpg


In [7]:
#print class_name unique values from the dataset
df['class_name'].value_counts()

class_name
Animal_Abuse        5741
NOT_ANIMAL_ABUSE    3480
Name: count, dtype: int64

In [8]:
import cv2
import os

path = 'dataset/'
images = []
labels = []

for i in os.listdir(path):
    data_path = os.path.join(path, str(i))
    filenames = [i for i in os.listdir(data_path)]

    for f in filenames:
        img_path = os.path.join(data_path, f)
        img = cv2.imread(img_path)

        if img is not None:
            images.append(img)
            labels.append(i)
        else:
            print(f"Error loading image: {img_path}")

In [10]:
print(len(images))

9221


In [11]:
images = np.array(images)
images = images.astype('float32')/255.0
images.shape

(9221, 224, 224, 3)

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

y = np.array(df['class_name'].values)

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)


[0 0 0 ... 1 1 1]


In [13]:
y = y.reshape(-1, 1) # Reshape to 2D array
onehotencoder = OneHotEncoder()  # Create the OneHotEncoder without categorical_features
Y = onehotencoder.fit_transform(y).toarray() # Convert to onehot
y.shape, Y.shape # Check the shape

((9221, 1), (9221, 2))

In [14]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images, Y, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(images, Y,test_size=0.2)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(7376, 224, 224, 3)
(7376, 2)
(1845, 224, 224, 3)
(1845, 2)


In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, CSVLogger

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', save_best_only=True)
tensorboard_callback = TensorBoard(log_dir='logs')  # Specify the log directory
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7)
csv_logger = CSVLogger('training_log.csv')

# Fit the model with callbacks
model.fit(train_x, train_y, epochs=50, batch_size=8, validation_data=(test_x, test_y),
          verbose=1, callbacks=[early_stopping, model_checkpoint_callback, tensorboard_callback, reduce_lr_callback, csv_logger], shuffle=True)

Epoch 1/50
 56/922 [>.............................] - ETA: 1:42:40 - loss: 0.6842 - accuracy: 0.5781

KeyboardInterrupt: 